In [1]:
import pandas as pd
import os
import numpy as np
import pyodbc
import datetime as dt
import warnings
warnings.filterwarnings('ignore')

In [2]:
class Load:
    
    def __init__(self , path = r'C:\Users\snappfood\Desktop\financial\23-29april' , excel_name = 'Payment 20220423 - 20220429.xlsx'):
        self.path = path
        self.excel_name = excel_name
        
    def load(self):
        data = pd.read_excel(f'{self.path}/{self.excel_name}',sheet_name=None)
        for name in data.keys():
            data[name].rename(str.lower, axis='columns', inplace=True)
            
        return data       

In [3]:
load_data = Load(path=r'C:\Users\snappfood\Desktop\financial\28may - 3jun' , excel_name = 'Payment 20220528 - 20220603.xlsx')
data = load_data.load()

## 1. Preprocessing

In [4]:
class Preprocess:
    
    def __init__ (self , data = data , output = r'C:\Users\snappfood\Desktop\financial\23-29april\output'):
        
        self.data = data
        self.output = output
        
    def wrong_shaba(self):
        ''' Detect all the wrong sheba numbers in excel file '''
        counter = 0
        wrong_sheba_tehran=pd.DataFrame()
        for i in range(data['tehran'].shape[0]):
            if len(str(data['tehran']['sheba number'].iloc[i]))!=26:
                wrong_sheba_tehran=wrong_sheba_tehran.append(data['tehran'].iloc[i])
                counter+=1
        self.wrong_sheba_tehran = wrong_sheba_tehran
        
        if counter!=0:
            print(f'There was {counter} incorrect sheba number(s)')
            wrong_sheba_tehran.to_excel(f'{self.output}/wrong_sheba_tehran.xlsx')  
            
        return counter , self.wrong_sheba_tehran
    
    def sheba_merge(self):
        '''merged datafarme which gained from wrong_sheba function and if it has a identical match in orginal excel file
        detect their biker ids'''
        
        wrong_sheba=self.wrong_sheba_tehran[['biker id','sheba number','lastname','total payable']]
        noSheba=self.data['no sheba'][['biker id','sheba number','lastname','total payable']]
        
        wrong_sheba.fillna(0,inplace=True)
        noSheba.fillna(0, inplace=True)
        
        #Full join on biker_id
        #___________________________________________________________________________#
        no_sheba_merged=pd.merge(wrong_sheba,noSheba,on='biker id',how='outer')
        #___________________________________________________________________________#
        
        missed_no_sheba = pd.DataFrame()
        #change if command:
        if no_sheba_merged['sheba number_x'].isnull().sum()== 0 and no_sheba_merged['sheba number_y'].isnull().sum()== 0:
            print('wrong sheba numbers are the same in both files')
        else:
            missed_no_sheba=missed_no_sheba.append(no_sheba_merged[no_sheba_merged['sheba number_y'].isnull()])
            missed_no_sheba=missed_no_sheba.append(no_sheba_merged[no_sheba_merged['sheba number_x'].isnull()])
            print(f'There was {missed_no_sheba.shape[0]} missed sheba numbers')
            return  missed_no_sheba
        
        #return no_sheba_merged
        
    def dublicate(self , attributes =['sheba number' , 'biker id' , 'cellphone']):
        ''' found any duplicated values in the selected futures and return their excel files'''
 
        for feature in attributes:

            df = self.data['tehran'][feature].dropna(axis = 0) 
            duplicated = pd.DataFrame()
            if df.unique().shape[0] == df.shape[0]:
                print(f'No duplicated {feature} founded')
            else:
                filt = df.duplicated(keep=False)==True
                duplicated= self.data['tehran'].loc[list(df[filt].index)]
                duplicated.to_excel(f'{self.output}/duplicated_{feature}.xlsx')
                print(f'There was {duplicated[feature].unique().shape[0]} duplicated values in the {feature}')
                        
               # values=df.value_counts().to_dict()
                #keys=list(values.keys())
                #for i in range(len(values)):
                 #   if values[keys[i]]>1:
                  #      duplicated =duplicated.append(data['tehran'][data['tehran'][feature]==keys[i]])
                #duplicated.to_excel(f'{self.output}/duplicated_{feature}.xlsx')
                  
    def payment_comp(self):
        '''check the positive_payable datas with payment datas '''
        
        pos_payable=data['tehran'][data['tehran']['total payable']>0]
        neg_payable=data['tehran'][data['tehran']['total payable']<=0]
        self.neg_payable = neg_payable.copy()
        
        pos_payable_correct_sheba =pos_payable[~pos_payable['biker id'].isin(data['no sheba']['biker id'])]
        tehran_positive_payable=pos_payable_correct_sheba[['biker id', 'total payable', 'lastname','sheba number']]
        
        pay_data =data['payment'][['biker id', 'total payable', 'lastname','firstname' ,'sheba number']]  
        pay_comp = pd.merge(tehran_positive_payable, pay_data, on='biker id', how='left')
        
        if pay_comp['sheba number_y'].isnull().sum()!=0:
            
            missed_biker_from_payment =pay_comp[pay_comp['sheba number_y'].isnull()]
            missed_biker_from_payment.to_excel(f'{self.output}/missed_in_payment_bikers.xlsx')
        else:
            print('biker payments in tehran and payment sheets are the same')
            
 
                
        tehran_comp_data=data['tehran']
        noSheba_comp_data=data['no sheba']
        payment_comp_data=data['payment']
        #positive payments and valid sheba number
        filt = (tehran_comp_data['total payable']>0) & (~tehran_comp_data['biker id'].isin(list(noSheba_comp_data['biker id'])))
        tehran_comp_data[filt]
        
        self.tehran_comp_data=tehran_comp_data[filt][['biker id', 'sheba number','total payable','firstname','lastname','express trips']]
        self.payment_comp_data=payment_comp_data[['biker id', 'sheba number','total payable','firstname','lastname','express trips']]
               
        
        return self.tehran_comp_data , self.payment_comp_data , self.neg_payable 
      
    

In [5]:
load_data = Load(path=r'C:\Users\snappfood\Desktop\financial\28may - 3jun' , excel_name = 'Payment 20220528 - 20220603.xlsx')
data = load_data.load()
preprocess = Preprocess(data=data , output=r'C:\Users\snappfood\Desktop\financial\28may - 3jun\output' )
counter , wrong_sheba_tehran  = preprocess.wrong_shaba()
no_sheba_merged  = preprocess. sheba_merge()
dublicate = preprocess . dublicate()
tehran_comp , payment_comp , neg_payable  = preprocess.payment_comp()

There was 208 incorrect sheba number(s)
wrong sheba numbers are the same in both files
No duplicated sheba number founded
No duplicated biker id founded
No duplicated cellphone founded
biker payments in tehran and payment sheets are the same


## 2.Check with DataBase

In [6]:
class Comparison(Preprocess):
    
    def __init__(self ,  start_date , end_date ,data = data ,neg_payable = neg_payable ,output = r'C:\Users\snappfood\Desktop\financial\23-29april\output' ):
        
        self.output = output
        self.data=data
        self.neg_payable = neg_payable
        self.start_date = start_date
        self.end_date = end_date
        
    def connection(self):
        server = '79.175.132.125,1410'
        database = 'express_db'
        username = 'ExpressAnalyst'
        password = '^cVGXN6q[nY#[-GE'
        Driver = 'SQL Server'
        cnxn = pyodbc.connect(
                'DRIVER={SQL Server};SERVER=' + server + ';DATABASE=' + database + ';UID=' + username + ';PWD=' + password)
        cursor = cnxn.cursor()
        return cursor 

    def database_check(self):
        '''check the obtained datas with databse'''

        query_1=f"""SELECT user_id,
           count(distinct id) as tedad
            from  express_db..trip
            where trip_status like '%Delivered%'
              and created_at >= '{self.start_date}'
              and created_at < '{self.end_date}'
              and user_id <> 1
              -- and t.city_id = 1

            group by user_id"""

        cursor=self.connection()
        cursor.execute(query_1)
        online_data=cursor.fetchall()  
        online_df = pd.DataFrame.from_records(online_data , columns = ['biker id' , 'db_trips'])
        online_df.to_csv(f'{self.output}/database_data.csv') 
        self.online_df = online_df.copy()
        
        #filtered negative payables and incorrect shebas from DB datas:
        
        edited_df=online_df[~online_df['biker id'].isin(list(data['no sheba']['biker id']))]
        edited_df=online_df[~online_df['biker id'].isin(list(self.neg_payable['biker id']))]
        self.edited_df= edited_df.copy()
        
        Payments=self.data['payment'][['biker id', 'express trips']]
        bikers_no_payment=pd.merge(self.edited_df,Payments,on='biker id',how='left')
        
        self.bikers_no_payment = bikers_no_payment.copy()
        
        bikers_no_payment.to_csv(f'{self.output}/bikers.csv')
        
        missed_biker_from_file = bikers_no_payment[bikers_no_payment['express trips'].isnull()]

        
        return self.online_df , self.edited_df , self.bikers_no_payment , missed_biker_from_file
    
    
    def diff_trips(self):
        '''this function retrive payment sheet of excel and database table to check number of trips in both and compare them '''
        
        self.bikers_no_payment.fillna(0, inplace=True)
        
        self.bikers_no_payment['diff']=abs(self.bikers_no_payment['express trips']-self.bikers_no_payment['db_trips'])
        
        self.bikers_no_payment=self.bikers_no_payment[~self.bikers_no_payment['biker id'].isin(list(self.data['no sheba']['biker id']))]
        self.bikers_no_payment=self.bikers_no_payment[~self.bikers_no_payment['biker id'].isin(list(self.neg_payable['biker id']))]
        
        
        self.bikers_no_payments=self.bikers_no_payment[self.bikers_no_payment['diff']>4].sort_values(by=['biker id'])
        
        #save:
        self.bikers_no_payments.to_excel(f'{self.output}/Diff_Trips_DB.xlsx')

        
        return self.bikers_no_payments
    
    
    def diff_date(self):
        self.biker_id_list = self.bikers_no_payments['biker id'].to_list()
        self.biker_id_list.remove(20008)
        
         
        start_date = dt.datetime.strptime(self.start_date , '%Y-%m-%d')
        created_date = start_date - dt.timedelta(1)    
       
        delivery_date = start_date 
        
        delivery_date = str(delivery_date.date())
        
        query_2 = f'''select t.id,
                       t.user_id,
                       t.created_at,
                       t.delivered_at,
                       t.trip_status,
                       t.zf_order_id
                        from express_db..trip t
                        where t.created_at >= '{created_date}'
                          and t.delivered_at >= '{self.start_date}'
                          and t.created_at < '{self.end_date}'
                        --and t.city_id = 1
                          and t.user_id  in {tuple(self.biker_id_list)}
                          and cast(t.created_at as date) <> cast(t.delivered_at as date)
                        ---order by t.created_at '''


        cursor=self.connection()
        cursor.execute(query_2)
        online_data=cursor.fetchall() 
        online_df = pd.DataFrame.from_records(online_data , columns = ['id' , 'biker_id','created_at' , 'deliverd_at' , 'trip_status' , 'zf_order_id'])
        online_df['allowed_diff'] = online_df['deliverd_at']-online_df['created_at'] < dt.timedelta(1)
        #just start and end of interval 
        end_date = dt.datetime.strptime(self.end_date , '%Y-%m-%d')
        start_date = dt.datetime.strptime(self.start_date,'%Y-%m-%d')
        
        filt=(online_df['created_at'].apply(lambda x: x.date())==created_date.date()) | (online_df['created_at'].apply(lambda x: x.date())==end_date.date()-dt.timedelta(1)) & (online_df['allowed_diff']==True)
        
        online_df=online_df[filt]
        new_diff =  self.bikers_no_payments.copy() 
        
        
        if online_df.shape[0]!=0:
            print(f'There are {online_df.shape[0]} trips with edge of interval travels ')
            
         
        for i in online_df['biker_id'].unique():
            filt = new_diff['biker id'] == i
            filt1 = online_df['biker_id'] == i
            n = online_df[filt1].shape[0]
            ind = new_diff[filt].index[0]
            new_diff.loc[ind , 'diff'] -= n  
        
        self.new_diff = new_diff
        return online_df , self.new_diff
    
    def multi_delivered_check(self):
        
        final = self.new_diff.copy()
        query_3 = f"""select tsh.trip_id,
               t.user_id,
               t.trip_status,
               tsh.to_status,
               t.city_id , 
               tsh.from_biker_id
            from express_db..TripStatusHistory tsh
                     join express_db..trip t
                          on t.id = tsh.trip_id
            where tsh.created_at >= '{self.start_date}'
              and tsh.created_at < '{self.end_date}'
              and tsh.from_biker_id in {tuple(self.biker_id_list)}
            -- and t.city_id=1
            group by tsh.trip_id, t.user_id, t.trip_status, t.city_id , tsh.from_biker_id , tsh.to_status
            having count(iif(tsh.to_status like '%delivered%', tsh.id, null)) > 0 """
        
        cursor=self.connection()
        cursor.execute(query_3)
        online_data=cursor.fetchall() 
        f_df = pd.DataFrame.from_records(online_data, columns =['trip id' , 'user id' , 'status', 'status tsh','city id' ,'biker id tsh' ])
        all_df = f_df.copy()
        #detect number of trips which deliverd with muliple bikers
        filt = all_df['user id'] != all_df['biker id tsh'] 
        x= all_df[filt]
        x = x.groupby(['biker id tsh'])['status'].count()
        
            
        #Decrease values from the original dataframe
        
        for i in x.index:
            filt = final['biker id'] == i
            ind = final[filt].index[0]
            n = x[i]
            final.loc[ind , 'diff'] -= n
            
        
        #delete 20008 biker id:
        filte = final['biker id'] == 20008
        index= final[filte].index[0]
        final.drop(index , axis = 0 , inplace=True)
        
        
        
        return all_df , final , x 
    

        
              

In [7]:
#date from sat to sat
comparison = Comparison(output=r'C:\Users\snappfood\Desktop\financial\28may - 3jun\output' , data=data , start_date = '2022-05-28' , end_date = '2022-06-04')
df , edited_df , bikers_no_payment ,missed_biker_from_file  = comparison.database_check()
bikers_no_payments = comparison.diff_trips()
df , new_diff = comparison.diff_date()
all_df, final , x = comparison.multi_delivered_check()



There are 11 trips with edge of interval travels 


In [16]:
final

,biker id,db_trips,express trips,diff
868,1314,263,268.0,0.0
5437,1476,340,345.0,1.0
3268,4827,303,308.0,0.0
71,13735,309,314.0,1.0
4310,20028,297,302.0,2.0
3117,22158,365,370.0,1.0
3536,26516,234,239.0,1.0
297,28957,312,319.0,0.0
2682,42242,184,189.0,1.0
677,46453,228,233.0,0.0


## Financial Check

In [8]:
class Finance(Comparison):
    def __init__(self , data = data ,path=r'E:/Weekly Payments/tasvieh_file/' , name='tasvieh'   ):
        
        self.path = path
        
        self.data = data
        self.name = name
        if self.name[-4:]!= 'xlsx':
            self.name = name + '.xlsx'
        
    
    def calculation(self):
        
        f_df=pd.read_excel(f'{self.path}/{self.name}' , sheet_name ='pattern-1' , header= 1 )
        
        self.tasvieh_data=f_df.copy(deep=True)
        
        self.tasvieh_data.rename(columns={'مبلغ':'payable','نام و نام خانوادگی ذینفع':'name','شبای مقصد':'sheba number' }, inplace=True, errors='raise')
        self.tasvieh_data.drop(columns=['شماره سپرده ی مبدا','شماره مشتری','کد بانک مقصد','شرح تراکنش','تاریخ ارسال','شناسه واریز(اختیاری)'], inplace=True, errors='raise')
        
        self.payment_comp = self.data['payment']
        #Totman to rial
        self.payment_comp['total payable'] = payment_comp['total payable']*10
        #sort values by sheba:
        self.tasvieh_data.sort_values(by='sheba number', inplace=True)
        self.payment_comp.sort_values(by='sheba number', inplace=True)
        self.tasvieh_data.reset_index(drop=True, inplace=True)
        self.payment_comp.reset_index(drop=True, inplace=True)
        if self.tasvieh_data.shape[0] == self.payment_comp.shape[0]:
            print('Same number of sheba in both files')
    
        
        
        self.tasvieh_data['concated']=self.tasvieh_data['sheba number'].apply(str)+'_'+self.tasvieh_data['name'].apply(str)+'_'+self.tasvieh_data['payable'].apply(str)
        
        
        self.payment_comp['concated']=self.payment_comp['sheba number'].apply(str)+'_'+self.payment_comp['firstname'].apply(str)+' '+self.payment_comp['lastname'].apply(str)+'_'+self.payment_comp['total payable'].apply(str)
        
        tas_comp=self.tasvieh_data[['sheba number','name' , 'payable','concated']]
        pay_comp=self.payment_comp[['sheba number','lastname' ,'firstname', 'total payable','concated' ]]
        
        megered_tasvieh = pd.merge(tas_comp,pay_comp, on='sheba number', how='outer')
        
        
        if megered_tasvieh['payable'].sum()/ megered_tasvieh['total payable'].sum() == 1  or megered_tasvieh['payable'].sum()/ megered_tasvieh['total payable'].sum()==10:
            print('sum of pay is equal in both files')
        #------------------------------------------check name and lastname--------------------------------------#
        
        #if megered_tasvieh[megered_tasvieh['lastname'].isnull()]:
         #   ind = list(merged[merged['lastname'].isnull()].index)
          #  merged['lastname'][ind] = merged['name'][ind]
        filt = megered_tasvieh['concated_x'] != megered_tasvieh['concated_y']
        final_check = megered_tasvieh[filt]
        

        return megered_tasvieh, self.tasvieh_data , final_check
    
    '''def calculation(self):
        
        #length:
        if self.tasvieh_data.shape[0] == self.payment_comp.shape[0]:
            print('Same number of sheba in both files')
            

        
        self.tasvieh_data['concated']=self.tasvieh_data['sheba number'].apply(str)+'_'+self.tasvieh_data['name'].apply(str)+'_'+self.tasvieh_data['payable'].apply(str)
        self.payment_comp['concated']=self.payment_comp['sheba number'].apply(str)+'_'+self.payment_comp['lastname'].apply(str)+'_'+self.payment_comp['rial_payable'].apply(str)
        
        return self.tasvieh_data , self.payment_comp '''
        
        
        
        

In [9]:
finance = Finance(data= data , path =r'C:\Users\snappfood\Desktop\financial\28may - 3jun' , name = 'tasvieh.xlsx')
merged,f_df , final_check= finance.calculation()


Same number of sheba in both files
sum of pay is equal in both files


In [10]:
final_check

,sheba number,name,payable,concated_x,lastname,firstname,total payable,concated_y
135,IR040170000000336758144007,بهمن ایپکچی,9790000,IR040170000000336758144007_ بهمن ایپکچی_9790000,بهمن ایپکچی,NaN,9790000,IR040170000000336758144007_nan بهمن ایپکچی_979...
836,IR150190000000212739320000,محمد میلاد نوذر,4415000,IR150190000000212739320000_ محمد میلاد نوذر_44...,محمد میلاد نوذر,NaN,4415000,IR150190000000212739320000_nan محمد میلاد نوذر...
1152,IR200700001000113310565001,مهدی بداقی,3000000,IR200700001000113310565001_ مهدی بداقی_3000000,مهدی بداقی,NaN,3000000,IR200700001000113310565001_nan مهدی بداقی_3000000
1615,IR280540100247000188993609,قربان محمد قزل,10787000,IR280540100247000188993609_ قربان محمد قزل_107...,قربان محمد قزل,NaN,10787000,IR280540100247000188993609_nan قربان محمد قزل_...
2377,IR410120010000005825701150,مسعود مناهی,20945000,IR410120010000005825701150_ مسعود مناهی_20945000,مسعود مناهی,NaN,20945000,IR410120010000005825701150_nan مسعود مناهی_209...
2804,IR480120010000005033073866,سید علی بزرگی دار شیرخانی,25462000,IR480120010000005033073866_ سید علی بزرگی دار ...,سید علی بزرگی دار شیرخانی,NaN,25462000,IR480120010000005033073866_nan سید علی بزرگی د...
3177,IR540160000000000826576699,مسعود بزرگی,18300000,IR540160000000000826576699_ مسعود بزرگی_18300000,مسعود بزرگی,NaN,18300000,IR540160000000000826576699_nan مسعود بزرگی_183...
3231,IR540570270110011607819001,امیر یاری,29005000,IR540570270110011607819001_ امیر یاری_29005000,امیر یاری,NaN,29005000,IR540570270110011607819001_nan امیر یاری_29005000
3412,IR570610000000400814053548,علی باقری اصل,1455000,IR570610000000400814053548_ علی باقری اصل_1455000,علی باقری اصل,NaN,1455000,IR570610000000400814053548_nan علی باقری اصل_1...
3544,IR600130100000000055755434,جعفر سنگ سفیدی,10098000,IR600130100000000055755434_ جعفر سنگ سفیدی_100...,جعفر سنگ سفیدی,NaN,10098000,IR600130100000000055755434_nan جعفر سنگ سفیدی_...
